<a href="https://colab.research.google.com/github/sboomi/exploradome_tangram/blob/tf2---team-3/Tangram_inceptionV3_dataFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image classification

Our objective is to classify tangram from images. We are going to build a deep learning with library TensorFlow2.  


## Step 1 : Import packages

Let's start by importing the required packages. The `os` package is used to read files and directory structure, NumPy is used to convert python list to numpy array and to perform required matrix operations and `matplotlib.pyplot` to plot the graph and display images in the training and validation data.

Import Tensorflow and the Keras classes needed to construct our model.

In [ ]:
import tensorflow as tf
import keras
import matplotlib.image as img
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import collections
from collections import defaultdict
from shutil import copy, copytree, rmtree

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras

import cv2
import time
import os
import random

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import itertools


# Nouvelle section

## Step 2 :  Load data
We assign variables with the proper file path for the training, validation set and testing.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
PATH = '/content/drive/My Drive/data/final_dataset_balanced'
train_dir = os.path.join(PATH, 'train')
test_dir = os.path.join(PATH, 'test')




## Step3 : Data pre-processing and data augmentation
we will "augment" our training via a number of random transformations, so that our model would never see twice the exact same picture. This helps prevent overfitting and helps the model generalize better.
1. We will use the keras.preprocessing.image.ImageDataGenerator class. This class allows to configure random transformations and normalization operations to be done on your image data during training 
2. We will use .flow_from_directory() to generate batches of image data (and their labels) directly from our jpgs in their respective folders, applies rescaling, and resizes the images into the required dimensions.


In [ ]:
n_classes = 12
IMAGE_SIZE = (299, 299)
batch_size = 31

# Our original images consist in RGB coefficients in the 0-255, but such values would be too high for our models to process, so we target values between 0 and 1 instead by scaling with a 1/255.
# width_shift_range, height_shift_range: (fraction of total height||width). Range for random horizontal shifts.
# shear_range is for randomly applying shearing transformations (deformation from an angle)
# fill_mode: points outside the boundaries of the input are filled according to the given mode
# rotation_range: degree range for random rotations.

test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
        rotation_range=50,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2, 
        horizontal_flip=True,
        fill_mode='nearest', validation_split=0.3)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode='categorical', subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode='categorical', subset='validation')

# class_mode: set “binary” if you have only two classes to predict, if not set to“categorical” (her we have 12 classes to predict)

## Step 4 : Create the model
There are different ways to modulate entropic capacity. The main one is the choice of the number of parameters in your model, i.e. the number of layers and the size of each layer. To reduce  overfitting we wille use dropout. 
we would use an optimizer with a very slow learning rate. In general, SGD is good choice for this as opposed to adaptive methods like Adam etc.
  



In [ ]:
# create the base pre-trained model
inception = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = inception.output
x = GlobalAveragePooling2D()(x)
#  let's add a fully-connected layer
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)
# we have 12 classes
predictions = Dense(12,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

In [ ]:
checkpointer = ModelCheckpoint(filepath='best_model_inceptionV3.h5', verbose=1, save_best_only=True)
# model weights are saved at the end of every epoch, if it's the best seen

csv_logger = CSVLogger('history_accuracy.log')
# CSVLogger:  streams epoch results to a CSV file.

callbacks=[csv_logger, checkpointer]
# list of CSVFile and weights to stock the value to stop the training

In [ ]:
# this is the model we will train
model = Model(inputs=inception.input, outputs=predictions)

In [ ]:

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])




### Model summary

View all the layers of the network using the model's `summary` method:

In [ ]:
model.summary()

### Train the model

In [ ]:

history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    verbose=1,
    callbacks = [callbacks],
    workers=10)


# callbacks: list of CSVFile and weights to stock the value to stop the training (to avoid overfitting)
# workers: Sequence input only. Maximum number of processes to spin up when using process-based threading.

In [ ]:
# plot to show the evolution of accuracy and loss during epochs

def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(10,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'bo--', label = "accuracy")
    plt.plot(history.history['val_accuracy'], 'ro--', label = "val_accuracy")
    plt.title("train_accuracy vs val_accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

In [ ]:
plot_accuracy_loss(history)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255 ) 

In [ ]:
 # all the test images should be placed inside a separate folder inside the test folder. 

 # shuffle: False because we have to keep the order of the images
 # target_size: the size of our input images, every image will be resized to this size 
 
test_data_gen = test_datagen.flow_from_directory(batch_size=batch_size,
                                                              directory=test_dir,
                                                              shuffle=False,
                                                              target_size=IMAGE_SIZE,
                                                              class_mode='categorical')
                                                    

In [ ]:
pred=model.predict_generator(test_data_gen,verbose=1,steps=960/batch_size)

In [ ]:
# argmax to chose the greatest accuracy for the predicted class
predicted_class_indices=np.argmax(pred,axis=1)

# Keras gives indices to each classe
# to have good indice with the good name class
labels = (train_generator.class_indices)
print(labels)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
# to stock the informations about the prediction of each image into a dataframe

filenames=test_data_gen.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

In [ ]:
real_pred = []

for filename in filenames:
  real_pred.append(filename.split('/')[0])

In [ ]:
cm = confusion_matrix(real_pred, predictions, labels=["bateau", "bol", "chat", "coeur", "cygne", "lapin", "maison", "marteau", "montagne", "pont", "renard", "tortue"])

print(cm)

In [ ]:
accuracy_score(real_pred, predictions)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(9,9))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm=cm, classes=train_generator.class_indices, title='Confusion Matrix')